# Read Data

+ Data are stored in six fields (i.e. XMM, VVDS, HECTOMAP, GAMA09H, WIDE12H, GAMA15H)
+ Use column `hsc_y3_zbin` to select galaxies in four redshift bins

+ the column names and descriptions are:
    - i_ra, i_dec: RA and DEC of the object
    - object_id: object_id to be cross matched to get photometric redshifts.
    - i_hsmshaperegauss_e1/2:  Galaxy distortion (e1, e2) in sky coordinates
    - i_hsmshaperegauss_derived_sigma_e: per-component shape measurement uncertainty
    - i_hsmshaperegauss_derived_rms_e: per-component RMS ellipticity estimate
    - i_hsmshaperegauss_derived_weight: Weight used for lensing calculations
    - i_hsmshaperegauss_derived_shear_bias_m : Multiplicative bias factor 
    - i_hsmshaperegauss_derived_shear_bias_c1/2: Additive bias factor for each component
    - i_hsmshaperegauss_resolution: The resolution compared to the PSF 
    - i_apertureflux_10_mag: The flux in aperture with radius 1 arcsec 
    - hsc_y3_zbin: redshift bin number (from 1 to 4) for the HSC cosmic shear analysis.
    - b_mode_mask: Mask to remove regions with large B-mode (use b_mode_mask==True to select galaxies not in the masked region)

In [1]:
import os
import astropy.io.fits as pyfits
data_dir = "/work/xiangchong.li/work/S19ACatalogs/catalog_obs_reGaus_public"
file_name = os.path.join(data_dir, "XMM.fits")
data = pyfits.getdata(file_name)
data_z1 = data[data["hsc_y3_zbin"] ==1 ]

# Calculate the average multiplicative bias and shear response
The multiplicative bias includes
+ shear estimation bias (`mbias`)
+ selection bias (`msel`)

In additon, we need to get the fractional additive bias from selection (`asel`) and the shear response (`response`)

In [15]:
# We are estimating the average over all fields
data_z1 = []
data_dir = "/work/xiangchong.li/work/S19ACatalogs/catalog_obs_reGaus_calibrated/"
for fieldname in ["XMM", "VVDS", "HECTOMAP", "GAMA09H", "WIDE12H", "GAMA15H"]:
    file_name = os.path.join(data_dir, "%s_calibrated.fits" % fieldname)
    data = pyfits.getdata(file_name)
    sel = (data["hsc_y3_zbin"] ==1) & (data["b_mode_mask"])
    data_z1.append(data[sel])
data_z1 = np.hstack(data_z1)
wsum = np.sum(data_z1['i_hsmshaperegauss_derived_weight'])

# Multiplicative bias
mbias = np.sum(
    data_z1['i_hsmshaperegauss_derived_shear_bias_m']
    * data_z1['i_hsmshaperegauss_derived_weight']
) / wsum

# Shear response
response = 1 - np.sum(
    data_z1['i_hsmshaperegauss_derived_rms_e']**2.0
    * data_z1['i_hsmshaperegauss_derived_weight']
) / wsum

# Selection bias
from utils_shear_ana import catutil
msel, asel, msel_err, asel_err = catutil.get_sel_bias(
    data_z1['i_hsmshaperegauss_derived_weight'],
    data_z1['i_apertureflux_10_mag'],
    data_z1['i_hsmshaperegauss_resolution'],
)

# Transform the shape catalog to shear catalog

In [16]:
# shear
g1, g2 = catutil.get_shear_regauss(data_z1, mbias, msel, asel)
# position
ra, dec = catutil.get_radec(data_z1)
# weight
weight = catutil.get_shape_weight_regauss(data_z1)